Importing libraries

In [2]:
import pandas as pd
from google.cloud import vision, storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\storage_key.json"

Reading csv file containing product_name and url

In [4]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\data.csv")
data

,product_name,url
0,NIKE Womens W Revolution 7 Running Shoe,https://m.media-amazon.com/images/I/71cy1dbMh5...
1,Puma Men's T-Shirt,https://m.media-amazon.com/images/I/51V8KtLHdm...
2,Asus Vivobook 15,https://m.media-amazon.com/images/I/71Ue-ZFSPJ...
3,Brown Zip Wallet,https://m.media-amazon.com/images/I/910SYv5lcO...
4,Apple Iphone 15 pro max,https://m.media-amazon.com/images/I/81fxjeu8fd...


Storing url column in data_url

In [5]:
data_url = data['url']
data_url

0    https://m.media-amazon.com/images/I/71cy1dbMh5...
1    https://m.media-amazon.com/images/I/51V8KtLHdm...
2    https://m.media-amazon.com/images/I/71Ue-ZFSPJ...
3    https://m.media-amazon.com/images/I/910SYv5lcO...
4    https://m.media-amazon.com/images/I/81fxjeu8fd...
Name: url, dtype: object

Empty String variable to store results

In [6]:
txt = ""

Function to extract information from image

In [7]:
def extract_info(uri):
    global txt
    client = vision.ImageAnnotatorClient()
    img = vision.Image()
    img.source.image_uri = uri
    
    #For retriving Brand
    res_logo = client.logo_detection(img)
    logos = res_logo.logo_annotations
    logo = [log.description for log in logos]
    txt = txt + f"Brand: None, " if len(logo)==0 else txt + f"Brand: {logo[0]}, "
    
    #For retriving Labels
    res_label = client.label_detection(img)
    labels = res_label.label_annotations
    label = [lab.description for lab in labels]
    txt = txt + f"Description: None, " if len(label)==0 else txt + f"Description: {str(label)} ,"
    
    #For retriving Text
    res_text = client.document_text_detection(img)
    texts = res_text.text_annotations
    text = [t.description for t in texts]
    txt = txt + f"Text: None" if len(text)==0 else txt + f"Text: {text[0]}"
    

Creating new dataframe to save results

In [8]:
result = data
result

,product_name,url
0,NIKE Womens W Revolution 7 Running Shoe,https://m.media-amazon.com/images/I/71cy1dbMh5...
1,Puma Men's T-Shirt,https://m.media-amazon.com/images/I/51V8KtLHdm...
2,Asus Vivobook 15,https://m.media-amazon.com/images/I/71Ue-ZFSPJ...
3,Brown Zip Wallet,https://m.media-amazon.com/images/I/910SYv5lcO...
4,Apple Iphone 15 pro max,https://m.media-amazon.com/images/I/81fxjeu8fd...


Iterating through function to extract features of image and adding it to result dataframe

In [9]:
for i in range(len(data_url)):
    txt = ""
    extract_info(data_url[i])
    result.loc[i, "Extracted_Details_from_Image"] = txt
    

Resultant Dataframe

In [10]:
result

,product_name,url,Extracted_Details_from_Image
0,NIKE Womens W Revolution 7 Running Shoe,https://m.media-amazon.com/images/I/71cy1dbMh5...,"Brand: Nike, Description: ['Footwear', 'Shoe',..."
1,Puma Men's T-Shirt,https://m.media-amazon.com/images/I/51V8KtLHdm...,"Brand: Puma, Description: ['Head', 'Arm', 'Hum..."
2,Asus Vivobook 15,https://m.media-amazon.com/images/I/71Ue-ZFSPJ...,"Brand: Asus, Description: ['Computer', 'Laptop..."
3,Brown Zip Wallet,https://m.media-amazon.com/images/I/910SYv5lcO...,"Brand: Fastrack, Description: ['Brown', 'Recta..."
4,Apple Iphone 15 pro max,https://m.media-amazon.com/images/I/81fxjeu8fd...,"Brand: iPhone, Description: ['Mobile phone', '..."


Saving Dataframe to CSV file

In [11]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)

Uploading csv file to Cloud Stroage

In [12]:
storage_cli = storage.Client()
bucket = storage_cli.get_bucket("bucket-shreyash")
blob = bucket.blob("Assessment-2/Extracted_features_from_image.csv")
blob.upload_from_filename("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
print("completed")

completed
